 Importing neccessary modules

In [ ]:
%pip install pubchempy
%pip install pandas
%pip install beautifulsoup4
%pip install selenium

In [2]:
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from pubchempy import *
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait as Wait
from selenium.webdriver.support import expected_conditions as EC


Function for fetching elements in page 

In [3]:
def Fetch_Element(driver,element_id,fetch_duration):
    try:
        element = Wait(driver,fetch_duration).until(EC.presence_of_element_located((By.ID,element_id)))
        print(f"Element found: {element_id}")
        return element
    except:
        print(f"ELEMENT NOT FOUND: {element_id}")
        raise TimeoutError(f"Element with {element_id} not found.")

Scraping page with pagination

In [5]:
browser = webdriver.Chrome(executable_path='./chromedriver')
url = "https://www.fishersci.com/us/en/browse/80013995/tertiary-amines?categoryKey=80013995&page=1"
browser.get(url)
browser.maximize_window()

time.sleep(10)

id_base = 'qa_srch_res_img_thumb_{}'
frames = []
page_count = 1

next_url = "https://www.fishersci.com/us/en/browse/80013995/tertiary-amines?categoryKey=80013995&page={}"
for result_num in range(1,973):
#    banner_el = browser.find_element(By.ID,'close')
#    banner_el.click()
    search_result_img = Fetch_Element(browser, id_base.format(result_num),10) 

    """ Click on search result image """
    print(f"Clicking result image {result_num}")
    browser.execute_script('arguments[0].scrollIntoView();', search_result_img)
    search_result_img.click()

    time.sleep(3)
    
    """ Setting up BS, selecting table"""
    source = browser.page_source.encode('utf-8')
    soup = bs(source, 'html.parser')
    table = soup.find_all("table",class_="specs_data")
    
    try:
        print("Parsing Table")
        df = pd.read_html(str(table))
        frames.append(df)
        
    except ValueError:
        print("Parsing Error")
        pass

    browser.back()

    
    """ Pagination """ 
    if result_num%30 == 0:
        # Increment page count
        page_count += 1
        if page_count != 34:
            browser.get(next_url.format(page_count))
            time.sleep(1)
            print(f"Paginating to page {page_count}")
        #next_element = browser.find_element(By.CLASS_NAME,"icon_Chevron_Right") 
        #browser.execute_script('arguments[0].scrollIntoView();', next_element)
        #next_element.click()



<ipython-input-5-ecf8adee19fd>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path='./chromedriver')


Element found: qa_srch_res_img_thumb_1
Clicking result image 1
Parsing Table
Element found: qa_srch_res_img_thumb_2
Clicking result image 2
Parsing Table
Element found: qa_srch_res_img_thumb_3
Clicking result image 3
Parsing Table
Element found: qa_srch_res_img_thumb_4
Clicking result image 4
Parsing Table
Element found: qa_srch_res_img_thumb_5
Clicking result image 5
Parsing Table
Element found: qa_srch_res_img_thumb_6
Clicking result image 6
Parsing Table
Element found: qa_srch_res_img_thumb_7
Clicking result image 7
Parsing Table
Element found: qa_srch_res_img_thumb_8
Clicking result image 8
Parsing Table
Element found: qa_srch_res_img_thumb_9
Clicking result image 9
Parsing Table
Element found: qa_srch_res_img_thumb_10
Clicking result image 10
Parsing Table
Element found: qa_srch_res_img_thumb_11
Clicking result image 11
Parsing Table
Element found: qa_srch_res_img_thumb_12
Clicking result image 12
Parsing Table
Element found: qa_srch_res_img_thumb_13
Clicking result image 13
Parsi

## Extracting CID's from DataFrame

In [6]:
pubchem_cids = set()

num_of_compounds = len(frames)

for num in range(num_of_compounds):
    num_of_tables = len(frames[num])
    for num_table in range(num_of_tables):
        df = frames[num][num_table]
        idx_list = df.loc[df[0] == 'PubChem CID'].index.tolist()
        if len(idx_list) > 0:
            pubchem_cids.add(df.iloc[idx_list[0]][1])

print(f"Number of extracted pubChem ID's: {len(pubchem_cids)}")




Number of extracted pubChem ID's: 513


In [7]:
immissable_compounds = {}

for cid in pubchem_cids:
    compound = Compound.from_cid(cid)
    xlogp = compound.xlogp

    if xlogp is not None and xlogp >= 1:
        immissable_compounds[cid] = compound
        
print(f"Immiscible compounds length: {len(immissable_compounds)}")

Immiscible compounds length: 331


In [8]:
xyz_imissable_compounds = {}
for cid in immissable_compounds:
    try:
        compound = Compound.from_cid(cid, record_type="3d")
        xyz_imissable_compounds[cid] = compound
    except:
        print(f'{cid} has no 3d record')

9962045 has no 3d record
66616716 has no 3d record
77709 has no 3d record
11061735 has no 3d record
16180236 has no 3d record
15412 has no 3d record
16202489 has no 3d record
59601002 has no 3d record
21918000 has no 3d record
53384285 has no 3d record
16153173 has no 3d record
75483 has no 3d record
20603493 has no 3d record
59757673 has no 3d record
6504535 has no 3d record
21881240 has no 3d record
16184079 has no 3d record
75991 has no 3d record
15365 has no 3d record
44630166 has no 3d record
102130343 has no 3d record
14049 has no 3d record
51358300 has no 3d record
51358306 has no 3d record
10952754 has no 3d record
81880 has no 3d record
14366516 has no 3d record
14227 has no 3d record
7624 has no 3d record


In [9]:
print(f"Length of immiscible compounds with 3D records: {len(xyz_imissable_compounds)}")

Length of immiscible compounds with 3D records: 302


Function definition: generatexyz(filename, dictionary)

In [10]:
def generatexyz(filename, dictionary):
    for cid in dictionary:
        immissable_compound = dictionary[cid]
        atoms = immissable_compound.to_dict(properties=['atoms'])['atoms']
        number_of_atoms = len(atoms)
        xlogp = immissable_compounds[cid].xlogp

        lines = []
        lines.append(f'{number_of_atoms}\n')
        lines.append(f'{cid} {xlogp}\n')

        for atom in atoms:
            element = atom['element']
            lines.append(f'{element}\t')
            x, y, z = atom['x'], atom['y'], atom['z']
            lines.append(f'{x}\t')
            lines.append(f'{y}\t')
            lines.append(f'{z}\n')
            

        with open(f'{filename}.xyz', 'a') as f:
            f.writelines(lines)

generatexyz('immiscible_compounds',xyz_imissable_compounds)
            
        

Immiscible compounds with only one Nitrogen atom query

In [11]:
nitrogen_xyz_immissable_compounds = {}

for cid in xyz_imissable_compounds:
    nitrogen_count = 0
    immissable_compound = xyz_imissable_compounds[cid]
    atoms = immissable_compound.to_dict(properties=['atoms'])['atoms']

    for atom_object in atoms:
        if atom_object['element'] == 'N':
            nitrogen_count += 1
    
    if nitrogen_count == 1:
        nitrogen_xyz_immissable_compounds[cid] = xyz_imissable_compounds[cid]
        
print(f"Length of nitrogen immiscible compounds: {len(nitrogen_xyz_immissable_compounds)}")
generatexyz('nitrogen_immiscible_compounds',nitrogen_xyz_immissable_compounds)

Length of nitrogen immiscible compounds: 130
